In [110]:
from bs4 import BeautifulSoup
import requests

In [111]:
def parser(url):
    headers = headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = requests.get(url, headers).content
    soup = BeautifulSoup(content, "html.parser")

    return soup

In [112]:
def job_card(url):
    soup = parser(url)
    
    efc_jobcard = soup.find('div', {'class': 'd-flex flex-column jobs-container ng-star-inserted'}).find_all('efc-job-card')

    card_details = [card.find('efc-card-details') for card in efc_jobcard]
    card_footer = [card.find('efc-card-footer') for card in efc_jobcard]
        
    return card_details, card_footer

In [113]:
efc_url = "https://www.efinancialcareers.co.uk/jobs/in-United-Kingdom" 

jcard = job_card(efc_url)
cdets, cfter = jcard[0], jcard[1]

print(cdets)


[<efc-card-details _ngcontent-ng-c2876617635="" _nghost-ng-c3221173188=""><div _ngcontent-ng-c3221173188="" class="d-flex justify-content-between align-items-baseline ng-star-inserted"><div _ngcontent-ng-c3221173188="" class="title"><a _ngcontent-ng-c3221173188="" class="font-subtitle-3-medium job-title job-title-spacing" data-gtm-trackable="job" efclink="" href="https://www.efinancialcareers.co.uk/jobs-UK-London-CTO_-_FX_-_Electronic_Trading.id21200883" id="GgVa3a34IwY3M3Fp" target="_self" title="CTO - FX - Electronic Trading"> <h3 _ngcontent-ng-c3221173188="" class="font-subtitle-3-medium ellipsis-2-row ng-star-inserted">CTO - FX - Electronic Trading</h3><!-- --><!-- --><!-- --></a></div><div _ngcontent-ng-c3221173188="" class="view-job-desktop-container px-0"><a _ngcontent-ng-c3221173188="" class="btn btn-x-small btn-primary" data-gtm-trackable="job" efclink="" href="https://www.efinancialcareers.co.uk/jobs-UK-London-CTO_-_FX_-_Electronic_Trading.id21200883" target="_self" title="CT

In [114]:
print(cfter)

[<efc-card-footer _ngcontent-ng-c2876617635="" _nghost-ng-c2406377873=""><!-- --><div _ngcontent-ng-c2406377873="" class="d-flex flex-column flex-lg-row justify-content-between font-helper-text info-container ng-star-inserted"><div _ngcontent-ng-c2406377873="" class="d-flex align-items-center flex-wrap"><span _ngcontent-ng-c2406377873="" class="dot-divider-after ng-star-inserted">In-Office</span><span _ngcontent-ng-c2406377873="" class="dot-divider-after last-job-criteria ng-star-inserted">£200k - £250k</span><!-- --><!-- --><!-- --><!-- --></div><div _ngcontent-ng-c2406377873="" class="d-flex align-items-end meta-section flex-wrap flex-shrink-0"><efc-first-apply-label _ngcontent-ng-c2406377873="" _nghost-ng-c1529728554="" class="first-apply-label ng-star-inserted" id="firstApplyLabel"><efc-icon _ngcontent-ng-c1529728554="" _nghost-ng-c2533289357=""><span _ngcontent-ng-c2533289357="" class="ng-star-inserted" style="color: var(--color-core-neutral-silver);"><svg _ngcontent-ng-c253328935

In [56]:
job_position = [item.find('a').get('title')  for item in cdets][0]
company = [item.find('div', {'class': 'font-body-3'}).text.lstrip() for item in cdets][0]
job_link = [item.find('a').get('href')  for item in cdets][0]
location = [item.find('span', {'class': 'dot-divider'}).text for item in cdets][0]
pos_type = [item.find('span', {'class': 'dot-divider ng-star-inserted'}).text for item in cdets][0]

work_arr = [item.find('span', {'class': 'dot-divider-after ng-star-inserted'}).text for item in cfter][0]
salary = [item.find('span', {'class': 'dot-divider-after last-job-criteria ng-star-inserted'}).text for item in cfter][0]

posted_on = [item.find('span', {'class': 'font-helper-text dot-divider color-font-secondary ng-star-inserted'}).text.lstrip() for item in cfter][0]



print(job_position)
print(company)
print(job_link)
print(location)
print(pos_type)

print(work_arr)
print(salary)

print(posted_on)

SPV Accountant
Cameron Kennedy 
https://www.efinancialcareers.co.uk/jobs-UK-London-SPV_Accountant.id21200880
London, United Kingdom
Contract
Hybrid
GBP75000 - GBP85000 per annum + Bonus Benefits
13 minutes ago 
